# Libraries

In [17]:
import cv2 # opencv
import mediapipe as mp
import numpy as np
import math
mp_drawing = mp.solutions.drawing_utils # drawing utilities
mp_pose = mp.solutions.pose # pose estimation model

# Functions

In [ ]:
# This function calculates angle between three points of any body
def calculate_angle(a,b,c):

    # remember that each point has X,Y,Z values ([0],[1],[2])
    a = np.array(a) # First
    b = np.array(b) # Mid 
    c = np.array(c) # End 

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0: # Angle between 0 and 180 (max angle of an arm is 180 actually)
        angle = 360-angle

    return angle

# Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px">

In [ ]:
# Video feed
cap = cv2.VideoCapture(0) # This allow to use any camera connected. Number represents the camera

# Curl counter variables
counter = 0
stage = None # "up" or "down" part of the curl


# Setup meediapipe instance
# Accuracy level. Higher score higher confidence required for detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read() # frame is for each image captured

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try: # try because sometimes it could not be detected
            landmarks = results.pose_landmarks.landmark

            # Get coordinates

            # Curl
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y] 

            # Calculate angles for biceps contraction
            left_angle_bc = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_angle_bc = (left_angle_bc).round(2)

            right_angle_bc = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_angle_bc = (right_angle_bc).round(2)

            # Calculate angles for elbows sticked to the body
            left_angle_st = calculate_angle(left_hip, left_shoulder, left_elbow)
            left_angle_st = (left_angle_st).round(2)

            right_angle_st = calculate_angle(right_hip, right_shoulder, right_elbow)
            right_angle_st = (right_angle_st).round(2)

            # Visualize left angle bicep contraction
            cv2.putText(image, str(left_angle_bc),
                        # Set position of angle text
                        # 640 and 480 are the width and height of the camera image
                        tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA # Text and line style
                        )
            
            # Visualize right angle bicep contraction
            cv2.putText(image, str(right_angle_bc),
                        # Set position of angle text
                        # 640 and 480 are the width and height of the camera image
                        tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA # Text and line style
                        )
            
            # Visualize left angle sticked elbow
            cv2.putText(image, str(left_angle_st),
                        # Set position of angle text
                        # 640 and 480 are the width and height of the camera image
                        tuple(np.multiply(left_shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA # Text and line style
                        )
            
            # Visualize right angle sticked elbow
            cv2.putText(image, str(right_angle_st),
                        # Set position of angle text
                        # 640 and 480 are the width and height of the camera image
                        tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA # Text and line style
                        )
            
            # Curl counter logic

            # Down condition
            down_cond = left_angle_bc > 160 and right_angle_bc > 160
            # Contraction condition
            contract_cond_left = left_angle_bc < 30 and left_angle_st < 20
            contract_cond_right = right_angle_bc < 30 and right_angle_st < 20

            if down_cond:
                stage = "down"
            if (contract_cond_left) and (contract_cond_right) and stage == "down":
                stage = "up"
                counter += 1
                print(counter)

        except Exception as e: 
            print(e)
            pass

        # Render curl counter

        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        # Rep data
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(stage),
                    (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        


        # Render detections
        # results.pose_landmarks shows the position each individual point of the body
        # mp_pose.POSE_CONNECTIONS shows the connections between each individual point of the body
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        #print(results)


        cv2.imshow("Mediapipe Feed", image) # Show the image captured

        if cv2.waitKey(10) & 0xFF == ord("q"): # Press q to close video
            break

    cap.release()
    cv2.destroyAllWindows()